In [1]:
import numpy as np

# Load the structured array from the file
loaded_arr = np.load('/kaggle/input/aila-cases-graph-embeddings/graph_embeddings.npy', allow_pickle=True)

# Check if the loaded array matches the original array
for item in loaded_arr:
    print("ID:", item['id'])
    print("Array:")
    print(len(item['embedding'][0]))
    break

ID: KG_C12.csv
Array:
64


In [2]:
pip install networkx node2vec numpy scikit-learn pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
import numpy as np
from node2vec import Node2Vec
import networkx as nx

query_graph_df = pd.read_csv("/kaggle/input/aila-query-kg-dataset/KG_Q5.csv")
G = nx.Graph()
G.add_nodes_from(query_graph_df["n"].unique())
G.add_nodes_from(query_graph_df["m"].unique())
rel_edges = [(s, t, {"r_type": r}) for r, s, t in zip(query_graph_df["r_type"], query_graph_df["n"], query_graph_df["m"])]
G.add_weighted_edges_from(rel_edges)

node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4, weight_key=None)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
query_embedding = np.array([model.wv[node] for node in G.nodes()])

Computing transition probabilities:   0%|          | 0/21 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [00:00<00:00, 163.21it/s]


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = []
for item in loaded_arr:
    similarity = cosine_similarity(query_embedding, item['embedding'])
    max_values_per_row = np.max(similarity, axis=1)
    total_sum = np.sum(max_values_per_row)
    similarity_value = total_sum/len(similarity)
    similarities.append([item['id'].split("_")[1].split(".")[0], similarity_value])

similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
# print(similarities)

position = None
for idx, sublist in enumerate(similarities):
    if sublist[0] == "C54":
        position = idx
        break
print(position)

163


In [5]:
import pandas as pd

query_doc_relevance = pd.read_csv("/kaggle/input/preprocessed-aila-irled-data/AILA_Relevance_Preprocessed_Data.csv")

query_doc_relevance.head()

,Query_Name,Document_Name,Relevance
0,AILA_Q1,C168,0
1,AILA_Q1,C382,0
2,AILA_Q1,C428,0
3,AILA_Q1,C949,0
4,AILA_Q1,C2303,0


In [6]:
query_relevant = []
embedding_docs = []

for item in loaded_arr:
    embedding_docs.append(item['id'].split("_")[1].split(".")[0])

# print(embedding_docs)

for index, row in query_doc_relevance.iterrows():
    if row['Relevance']==1:
        if row["Document_Name"] in embedding_docs:
            query_relevant.append(row["Query_Name"].split("_")[1])

total_relevant = len(query_relevant)
query_relevant = list(set(query_relevant))

# print(total_relevant)

In [7]:
query_doc_only_relevant = query_doc_relevance[query_doc_relevance['Relevance']==1]

query_doc_only_relevant.head()

,Query_Name,Document_Name,Relevance
1192,AILA_Q1,C14,1
2274,AILA_Q1,C9,1
3076,AILA_Q2,C27,1
3676,AILA_Q2,C22,1
6033,AILA_Q3,C1,1


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

tp = 0

for query in query_relevant:
    
    query_graph_df = pd.read_csv("/kaggle/input/aila-query-kg-dataset/KG_" + query + ".csv")
    G = nx.Graph()
    G.add_nodes_from(query_graph_df["n"].unique())
    G.add_nodes_from(query_graph_df["m"].unique())
    rel_edges = [(s, t, {"r_type": r}) for r, s, t in zip(query_graph_df["r_type"], query_graph_df["n"], query_graph_df["m"])]
    G.add_weighted_edges_from(rel_edges)

    node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4, weight_key=None)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    query_embedding = np.array([model.wv[node] for node in G.nodes()])
    
    similarities = []
    for item in loaded_arr:
        similarity = cosine_similarity(query_embedding, item['embedding'])
        max_values_per_row = np.max(similarity, axis=1)
        total_sum = np.sum(max_values_per_row)
        similarity_value = total_sum/len(similarity)
        similarities.append([item['id'].split("_")[1].split(".")[0], similarity_value])

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    for index, row in query_doc_only_relevant.iterrows():
        if row["Query_Name"].split("_")[1]==query:
            for idx, sublist in enumerate(similarities[:25]):
                if sublist[0] == row["Document_Name"]:
                    tp += 1
                    break
    print(query)
    
print(tp)

Computing transition probabilities:   0%|          | 0/25 [00:00<?, ?it/s]

Generating walks (CPU: 3): 100%|██████████| 50/50 [00:00<00:00, 153.89it/s]


Q8


Computing transition probabilities:   0%|          | 0/9 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [00:00<00:00, 314.12it/s]


...

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 95.32it/s]


Q34


Computing transition probabilities:   0%|          | 0/43 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:00<00:00, 85.91it/s]


Q24
14


In [9]:
precision = tp/(len(query_relevant)*10)
recall = tp/total_relevant

print("Relevant documents retrieved overall: ", tp)
print("Precision: ", precision)
print("Recall: ", recall)

Relevant documents retrieved overall:  14
Precision:  0.03255813953488372
Recall:  0.14
